<a href="https://colab.research.google.com/github/Riccardo-Venturi/Tesi_Script_Colab/blob/main/Homography1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Obiettivo #3: Gestire le Uscite (la Sfida Finale)

Questo è un problema complesso, come hai detto tu.
> le uscite sono quelle più informative e critiche ma sono prese in verso speculare

**La strategia vincente è trattarle come un problema separato ma quasi identico.**

1.  **Non Mischiare:** Non mettere gli ingressi e le uscite nello stesso CSV master per l'omografia. Le distorsioni sono diverse.
2.  **Crea una Pipeline per le Uscite:** Duplica il tuo notebook di omografia. Chiamalo `Homography_Uscite.ipynb`.
3.  **Applica una Trasformazione Iniziale:** La primissima cosa da fare dopo aver caricato l'immagine `uscita.jpg` è **specchiarla orizzontalmente**.
    ```python
    # All'inizio, dopo aver caricato l'immagine
    img_uscita = cv2.imread(PERCORSO_USCITA)
    img_uscita_specchiata = cv2.flip(img_uscita, 1) # 1 = flip orizzontale
    # Ora usa `img_uscita_specchiata` per tutto il resto del processo
    ```
4.  **Esegui la Pipeline di Omografia:** Usa la pipeline che abbiamo appena perfezionato (con RANSAC adattivo e filtro dinamico) sulle immagini delle uscite specchiate. Questo dovrebbe allinearle correttamente con le radiografie (che sono "viste da dietro").
5.  **Crea un Dataset Separato:** Alla fine, avrai uno `unet_dataset_uscite.zip`.
6.  **Unisci i Dataset:** Ora hai due dataset di alta qualità: `ingressi.zip` e `uscite.zip`. Puoi unirli in un'unica grande cartella per il training. Il modello ora vedrà entrambi i tipi di danno.

**Perché trattarle come nuovi fori:** Unet non sa cosa sia "l'ingresso" o "l'uscita". Vede solo pixel. Dargli più esempi di danni, anche se da prospettive diverse (che noi abbiamo "corretto" con il flip), arricchisce la sua conoscenza e lo rende più robusto.

### Piano d'Azione per Oggi

1.  **Modifica la Cella 4** del tuo script di omografia per includere il **RANSAC Sweep**.
2.  **Modifica la Cella 5** per usare **soglie dinamiche** basate sui quantili (es. `quantile(0.8)`).
3.  **Esegui la pipeline** e controlla quanti fori "buoni" ottieni. L'obiettivo è arrivare ad almeno 40-45 su 64.
4.  Una volta che sei soddisfatto, **genera lo zip finale** e usalo per addestrare di nuovo il tuo modello U-Net. I risultati dovrebbero migliorare drasticamente grazie a un dataset più grande e comunque di alta qualità.
5.  Metti da parte il problema delle uscite per ora. Risolviamo prima l'allineamento degli ingressi. Un passo alla volta.


#· Sintesi operativa

Il notebook carica le scan ottiche e le corrispondenti scan NDT (radiografiche) di una stessa maschera, legge un master CSV dove per ciascun foro ci sono le coordinate ottiche ↔ NDT, stima la omografia migliore tra i due domini (scandagliando diversi threshold RANSAC), allinea la NDT sull’ottica via warpPerspective, estrae patch per ogni foro, costruisce le maschere ground‑truth (foro + danno) e impacchetta tutto in PNG/ZIP pronti per l’addestramento di una U‑Net di segmentazione. Le parti principali sono:
0  setup & import
1  path + parametri
2  load csv → src/dst
3  helper funzioni
4  H_glob + KNN
5  locale vs globale → overlay & quality.csv
6  patch512 generator
7  (opt) angle refine
8  (opt) z‑score scale
9  zip packaging

<details>
    Setup & prerequisiti – install e import.

    I/O & config – path ai CSV, immagini e cartelle di output.

    Funzioni geometriche – estrazione dei quattro cardinali di un’ellisse (“pipeline A”) o campionamento uniforme del contorno (“pipeline B”).

    Stima di H – cv2.findHomography(..., cv2.RANSAC, th) con grid search sui threshold, scelta di quello con inlier‑ratio migliore.

    Warp & debug overlay – allineamento NDT‑>ottico, overlay RGB per controllo visivo.

    Crop & patching – ritaglio centrato sul foro (450 × 450 px circa) e salvataggio PNG.

    Costruzione maschere GT – Otsu + morphology (erosione & apertura) per separare foro e danno.

    Packaging – zipfile.ZipFile di patch e maschere per upload/uso ML.

2 What you need

| column                  | how to compute                                               | comment                                |
| ----------------------- | ------------------------------------------------------------ | -------------------------------------- |
| `hole_id`               | the existing `hole_idx`                                      | stays 1 … 64                           |
| `optical_x` `optical_y` | `(x1 + x2) // 2`, `(y1 + y2) // 2` **from the ingresso CSV** | centre of the hole in the optical scan |
| `ndt_x` `ndt_y`         | same centre but **from the radio CSV**                       | centre of the hole in the NDT scan     |


In [ ]:
# %% 0
#@title 0 Setup
#**Prerequisites:**
# %% 0 – Setup & import
# - Installa dipendenze se mancano (opencv‑python‑headless, numpy, pandas, scikit‑image…)
# - Importa tutte le librerie usate dal resto del notebook
# - NON mettere altro codice: così gli import rimangono una sola volta
!pip install opencv-python numpy pandas
import cv2
import numpy as np
import pandas as pd
import os, zipfile
from pathlib import Path

from sklearn.neighbors import NearestNeighbors as nn
from skimage.metrics import structural_similarity as ssim

import csv, math
from tqdm.auto import tqdm
import matplotlib.pyplot as plt



#Il problema principale del tuo script è la **mancanza di un flusso di dati lineare**. Ci sono almeno tre strategie diverse per l'allineamento mescolate insieme:
##1.  **Omografia Globale Semplice:** Calcola una `H` per tutta l'immagine e la usa per tutto.
##2.  **Omografia Locale per Ogni Foro:** Calcola una `H` specifica per ogni foro usando i suoi vicini (k-NN). Molto più precisa ma complessa.
##3.  **Scaling Semplice:** Ignora l'omografia e si limita a riscalare le patch.
##

In [ ]:
# ===================================================================
# CELLA 1: CONFIGURAZIONE
# ===================================================================
print("--- [FASE 1] Configurazione del Processo ---")

import cv2, json, pathlib, numpy as np, pandas as pd
from tqdm.notebook import tqdm
import zipfile
#lavoriamo con ingressi che sono più facili da definitre
# --- 1. PARAMETRI DELLO SCAN (MODIFICA SOLO QUESTA SEZIONE) ---
SCAN_ID = "1A" # Esempio: "1A", "1B", etc.
OPTICAL_IMAGE_NAME = f"T0_90_{SCAN_ID}_ingresso.jpg"
NDT_IMAGE_NAME = f"Carbon Textile {SCAN_ID}.jpg"
OPTICAL_CSV_NAME = f"T0_90_{SCAN_ID}_ingresso_ordered_NEW.csv"
NDT_CSV_NAME = f"Carbon Textile {SCAN_ID}_ordered_NEW.csv"

# --- 2. PERCORSI (MODIFICA SE NECESSARIO) ---
DRIVE_BASE_PATH = pathlib.Path("/content/drive/MyDrive")
OPTICAL_IMAGE_PATH = DRIVE_BASE_PATH / "Scan_Orientate" / OPTICAL_IMAGE_NAME
NDT_IMAGE_PATH = DRIVE_BASE_PATH / "FileTesi/ScanzioneProviniUsura/Radio" / NDT_IMAGE_NAME
OPTICAL_CSV_PATH = DRIVE_BASE_PATH / "CSV_FINALI/INGRESSI_ORDINATI" / OPTICAL_CSV_NAME
NDT_CSV_PATH = DRIVE_BASE_PATH / "CSV_FINALI/Radio_first_350 holes" / NDT_CSV_NAME
OUTPUT_BASE_DIR = pathlib.Path(f"/content/Homography_Output_{SCAN_ID}")

# --- 3. PARAMETRI DI PROCESSO (PUOI TENERE I DEFAULT) ---
PATCH_SIZE = 704             # Dimensione finale delle patch
K_NEIGHBORS = 9              # N. di vicini per l'omografia locale
RANSAC_THRESHOLD = 3.0       # Tolleranza per findHomography
# Parametri del filtro di qualità
SHIFT_QUANTILE = 0.90        # Scarta il 10% peggiore per lo shift
SSIM_QUANTILE = 0.10         # Scarta il 10% peggiore per SSIM
MIN_INLIER_RATIO = 0.7       # Almeno il 70% dei vicini deve essere inlier

# --- Creazione directory di output --- VUOTE
(OUTPUT_BASE_DIR / "overlays").mkdir(parents=True, exist_ok=True)
(OUTPUT_BASE_DIR / "final_patches/images").mkdir(parents=True, exist_ok=True)
(OUTPUT_BASE_DIR / "final_patches/masks").mkdir(parents=True, exist_ok=True)

print(f"✅ Configurazione caricata per lo Scan ID: {SCAN_ID}")

| Caso foro                   | Funzione                    | Logica                                                                                                                 |
| --------------------------- | --------------------------- | ---------------------------------------------------------------------------------------------------------------------- |
| Foro ellittico ben definito | `four_cardinals(cnt)`       | Fit di un’ellisse via `cv2.fitEllipse` e calcolo dei 4 assi principali ([tutorialspoint.com][1], [docs.opencv.org][2]) |
| Foro irregolare / usurato   | `sample_contour(cnt, k=32)` | Uniform sampling del contorno per robustezza                                                                           |

[1]: https://www.tutorialspoint.com/how-to-fit-the-ellipse-to-an-object-in-an-image-using-opencv-python?utm_source=chatgpt.com "How to fit the ellipse to an object in an image using OpenCV Python?"
[2]: https://docs.opencv.org/4.x/de/d62/tutorial_bounding_rotated_ellipses.html?utm_source=chatgpt.com "Creating Bounding rotated boxes and ellipses for contours - OpenCV"


In [ ]:
# ===================================================================
# CELLA 2: CARICAMENTO DATI E CREAZIONE MASTER CSV
# ===================================================================
print("--- [FASE 2] Creazione Master CSV ---")

try:
    opt_df = pd.read_csv(OPTICAL_CSV_PATH)
    ndt_df = pd.read_csv(NDT_CSV_PATH)
except FileNotFoundError as e:
    print(f"❌ ERRORE: File non trovato. Controlla i percorsi e lo SCAN_ID nella Cella 1. Dettagli: {e}")
    raise e

assert opt_df.shape[0] == ndt_df.shape[0], f"Disallineamento nel numero di fori: Ottico={len(opt_df)}, NDT={len(ndt_df)}"

opt_df["optical_x"] = (opt_df.x1 + opt_df.x2) // 2
opt_df["optical_y"] = (opt_df.y1 + opt_df.y2) // 2
ndt_df["ndt_x"] = (ndt_df.x1 + ndt_df.x2) // 2
ndt_df["ndt_y"] = (ndt_df.y1 + ndt_df.y2) // 2

master_df = (opt_df[["hole_idx", "optical_x", "optical_y"]]
             .merge(ndt_df[["hole_idx", "ndt_x", "ndt_y"]], on="hole_idx")
             .rename(columns={"hole_idx": "hole_id"})
             .sort_values("hole_id"))

MASTER_CSV_PATH = OUTPUT_BASE_DIR / f"{SCAN_ID}_master.csv"
master_df.to_csv(MASTER_CSV_PATH, index=False)

print(f"✅ Master CSV creato in: {MASTER_CSV_PATH} ({len(master_df)} fori)")

#Strategie
<details>
| Strategia                                   | Quando usarla                                             | Come si costruiscono i punti                                                                                                                                                                                                                                    |
| ------------------------------------------- | --------------------------------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **A. Fit di ellisse + quattro “cardinali”** | Ovale regolare dovuto a prospettiva; vuoi codice semplice | 1) `cv2.fitEllipse` su ciascun contorno.<br>2) Ricavi centro (C), semiasse maggior/minore (a,b) e angolo θ.<br>3) Calcoli i 4 punti di estremità (sinistra, destra, alto, basso) applicando la rotazione.<br>4) Con questi 4 ⇄ 4 punti chiami `findHomography`. |
| **B. Campionamento uniforme**               | Contorni irregolari / profili con schegge                 | 1) Rishapi il contorno a (N,2).<br>2) Usa `np.linspace` per prendere es. 32 indici equidistanti su *entrambi* i contorni.<br>3) Rimodella in (N,1,2) `float32` e passa a `findHomography` (RANSAC 5 px).                                                        |


<details>| Situazione                             | Pipeline A (cardinali)      | Pipeline B (campionamento) |
| -------------------------------------- | --------------------------- | -------------------------- |
| Foro ellittico regolare                | ✅ sub-pixel                 | ✅ ma inutile               |
| Foro con bave locali                   | ✅ (fitEllipse attenua)      | ⚠ necessita pre-filtrare   |
| Foro parziale tagliato dal bordo patch | ⚠ rischio fitEllipse errato | ✅ se punti interni sono ok |


### Il Nuovo Codice per Generare le Maschere Corrette
<details>
Sostituisci la sezione `C` del tuo script di generazione dataset con questo. Questo è il cuore della soluzione.

```python
# --- C. Crea la maschera del danno (GROUND TRUTH Y) - VERSIONE 2.0 CORRETTA ---
rad_warp = cv2.warpPerspective(rad_raw, H_use, (wO, hO))
rad_patch = rad_warp[y1:y2, x1:x2]

# --- QUESTO È IL NUOVO CUORE ---
# 1. Normalizza il contrasto come prima
rad_patch_normalized = cv2.normalize(rad_patch, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)

# 2. PASSO 1: Trova tutto ciò che è luminoso (Foro + Danno)
#    Usiamo una soglia di Otsu che trova il valore ottimale automaticamente
soglia_otsu, mask_tutto_il_luminoso = cv2.threshold(rad_patch_normalized, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# 3. PASSO 2: Isola solo il "cuore" del foro usando l'erosione
#    Creiamo un "kernel" (una forma) per l'erosione. Più è grande, più aggressiva sarà l'erosione.
kernel_size = 15 # Gioca con questo valore (es. 10, 15, 20)
kernel = np.ones((kernel_size, kernel_size), np.uint8)
mask_solo_foro = cv2.erode(mask_tutto_il_luminoso, kernel, iterations=1)

# 4. PASSO 3: Sottrai il foro dal totale per ottenere SOLO IL DANNO
damage_mask = cv2.subtract(mask_tutto_il_luminoso, mask_solo_foro)

# (OPZIONALE MA CONSIGLIATO) Pulizia finale per rimuovere piccoli artefatti
# Rimuove piccoli puntini di rumore
kernel_pulizia = np.ones((3,3), np.uint8)
damage_mask = cv2.morphologyEx(damage_mask, cv2.MORPH_OPEN, kernel_pulizia)

# D. Salva i due file
filename = f"hole_{hole_id:04d}.png"
cv2.imwrite(str(IMAGES_DIR / filename), opt_patch)
cv2.imwrite(str(MASKS_DIR / filename), damage_mask)

# --- (OPZIONALE) AGGIUNGI QUESTO PER UN CHECK VISIVO ---
# Salva le maschere intermedie per capire cosa sta succedendo
# DEBUG_DIR = Path("/content/debug_masks"); DEBUG_DIR.mkdir(exist_ok=True)
# cv2.imwrite(str(DEBUG_DIR / f"{filename}_1_tutto.png"), mask_tutto_il_luminoso)
# cv2.imwrite(str(DEBUG_DIR / f"{filename}_2_solo_foro.png"), mask_solo_foro)
# cv2.imwrite(str(DEBUG_DIR / f"{filename}_3_risultato.png"), damage_mask)

| Sintomo (quello che vedi)                                                           | Causa più probabile                                                              | Dove intervenire                                                          |
| ----------------------------------------------------------------------------------- | -------------------------------------------------------------------------------- | ------------------------------------------------------------------------- |
| Ellisse ruotata ma **non centrata**                                                 | fitEllipse corretto → crop centrato *prima* del warp, non **dopo**               | usa il centro **dell’ellisse warppata** per ritagliare                    |
| Bordo ottico ok ma la radiografia forma un **“cuscino”** pentagonale (angoli scuri) | punti cardinali scelti bene, ma raggio NDT troppo piccolo ⇒ scala in H sbagliata | aumenta **MARGIN** (es. 200 → 300) così il warp ha spazio                 |
| Foro sembra **oblungo** solo in alto                                                | inlier ratio < 0,6 nella fallback B → RANSAC ha eliminato troppi punti           | abbassa la soglia RANSAC da 5 px a 3 px **e** campiona più punti (es. 60) |
| Corte bande nere diagonali agli angoli                                              | il warp ritaglia dove non c’è informazione                                       | dopo `warpPerspective` fai `cv2.copyMakeBorder(aligned, ⟂)` con nero      |


In [ ]:
#### Cella 3: Toolbox di Funzioni Helper

'''Qui definiamo tutte le funzioni di supporto. Nessuna esecuzione, solo definizioni.
'''
# ===================================================================
# CELLA 3: TOOLBOX DI FUNZIONI HELPER
# ===================================================================
from sklearn.neighbors import NearestNeighbors
from skimage.metrics import structural_similarity as ssim

def crop_patch(img, cx, cy, size):
    x1, y1 = int(cx - size//2), int(cy - size//2)
    return img[max(y1,0):y1+size, max(x1,0):x1+size]

def create_ground_truth_mask(rad_patch):
    normalized = cv2.normalize(rad_patch, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    _, mask_all = cv2.threshold(normalized, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    kernel_erosion = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (15, 15))
    mask_hole = cv2.erode(mask_all, kernel_erosion, iterations=1)

    mask_damage = cv2.subtract(mask_all, mask_hole)

    kernel_clean = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    mask_damage_clean = cv2.morphologyEx(mask_damage, cv2.MORPH_OPEN, kernel_clean)

    final_mask = np.zeros_like(rad_patch, dtype=np.uint8)
    final_mask[mask_hole > 0] = 1
    final_mask[mask_damage_clean > 0] = 2

    return final_mask

print("✅ Funzioni helper definite.")

def largest_cnt(img):
    g = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) if img.ndim==3 else img
    _,th = cv2.threshold(g,0,255,cv2.THRESH_OTSU)
    cnts,_ = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    return max(cnts, key=cv2.contourArea) if cnts else None

def ellipse_pts(cnt):
    (cx,cy),(MA,ma),ang = cv2.fitEllipse(cnt)
    a,b = MA/2, ma/2
    R   = np.deg2rad(ang); ca,sa = np.cos(R), np.sin(R)
    base= np.float32([[ a,0],[-a,0],[0,b],[0,-b]])
    rot = np.array([[ca,-sa],[sa,ca]], np.float32)
    return (rot @ base.T).T + (cx,cy)

def sample_by_angle(cnt, k=40):
    (cx,cy),_ = cv2.minEnclosingCircle(cnt)
    pts = cnt.reshape(-1,2)-[cx,cy]
    ang = np.arctan2(pts[:,1], pts[:,0])
    order = ang.argsort()
    pts = pts[order]+[cx,cy]
    idx = np.linspace(0,len(pts)-1,k,dtype=int)
    return pts[idx].astype(np.float32)

def make_mask(img):
    g = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) if img.ndim==3 else img
    _,th = cv2.threshold(g,0,255,cv2.THRESH_OTSU)
    return cv2.medianBlur(th,5)

def micro_refine(opt_patch, rad_patch, delta=2.0, step=0.1):
    """piccola rotazione ±delta° che minimizza lo XOR foro–foro"""
    h,w   = rad_patch.shape[:2]
    centre= (w/2, h/2)
    mask_o= make_mask(opt_patch)
    best_loss, best_ang, best_out = 1e12, 0.0, rad_patch
    for ang in np.arange(-delta, delta+step, step):
        M   = cv2.getRotationMatrix2D(centre, ang, 1.)
        rot = cv2.warpAffine(rad_patch, M, (w,h),
                             flags=cv2.INTER_LINEAR,
                             borderMode=cv2.BORDER_CONSTANT,
                             borderValue=0)
        loss= cv2.countNonZero(cv2.bitwise_xor(mask_o, make_mask(rot)))
        if loss < best_loss:
            best_loss, best_ang, best_out = loss, ang, rot
    return best_out, best_ang, best_loss

| Osservazione                                                                                                        | Diagnosi tecnica                                                                                                                                                                                                                                                                                           |
| ------------------------------------------------------------------------------------------------------------------- | ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| L’overlay mostra la radiografia girata rispetto all’ottica (bordi diagonali, ellisse “capovolta”).                  | Il file **Carbon Textile 1A.jpg** è stato acquisito con un’inclinazione globale diversa (angolo di plate o testa di scansione). Quando applichi i 4 punti “cardinali” dell’ellisse, l’ordine di quei punti (±a, ±b) **non corrisponde** all’ordine nell’ottica, quindi `findHomography` compensa ruotando. |
| `detH ≈ 3.09`, scaleX ≈ 1.80, scaleY ≈ 1.71: valori plausibili.<br>Ma l’ellisse ottica risultante è ancora obliqua. | La H ha risolto scala + shear, **ma** manca un pre-allineamento di rotazione globale, perciò il foro è centrato ma l’intero patch appare ruotato.                                                                                                                                                          |


| colonna    | cosa significa                                          | tipico range “buono” |
| ---------- | ------------------------------------------------------- | -------------------- |
| `detH`     | determinante di *H* (≈ area scale)                      | 0.1 – 10             |
| `scaleX/Y` | norma prima riga / seconda riga di *H* (zoom)           | 0.3 – 3              |
| `inlier`   | frazione punti inlier (metodo B)                        | ≥ 0.6                |
| `axesA/B`  | semi‑assi ottica (px)                                   | ≈ diametro foro      |
| `ok`       | 1 = patch accettata (regola empirica su scale + inlier) | —                    |


In [ ]:

# ===================================================================
# CELLA 4.4 (MODIFICATA): OMOGRAFIA LOCALE ADATTIVA - nuova da usare
# ===================================================================
print("--- [FASE 4] Calcolo Omografia Adattiva e Metriche ---")
# ... (caricamento dati iniziale, H_glob, knn, etc. come prima) ...
#--- Carica dati ---
coord_df = pd.read_csv(MASTER_CSV_PATH)
opt_img = cv2.imread(str(OPTICAL_IMAGE_PATH))
rad_img = cv2.imread(str(NDT_IMAGE_PATH), 0)
hO, wO = opt_img.shape[:2]

src_pts = coord_df[['ndt_x', 'ndt_y']].to_numpy(np.float32)
dst_pts = coord_df[['optical_x', 'optical_y']].to_numpy(np.float32)

# --- Calcola H Globale una volta ---
H_glob, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
knn = NearestNeighbors(n_neighbors=K_NEIGHBORS).fit(src_pts)

quality_metrics = []
ransac_thresholds_to_try = [1.0, 3.0, 5.0, 7.0] # Proveremo queste soglie

for i, row in tqdm(coord_df.iterrows(), total=len(coord_df), desc="Analizzando fori"):
    src0 = src_pts[i].reshape(1,-1)
    _, indices = knn.kneighbors(src0)
    indices = indices.flatten()

    best_H_loc = None
    best_inliers = -1

    # --- RANSAC SWEEP ---
    # Per ogni foro, proviamo diverse soglie RANSAC e teniamo la migliore
    for threshold in ransac_thresholds_to_try:
        H_loc_candidate, mask = cv2.findHomography(
            src_pts[indices], dst_pts[indices],
            cv2.RANSAC, threshold
        )

        current_inliers = mask.ravel().sum() if mask is not None else 0

        if current_inliers > best_inliers:
            best_inliers = current_inliers
            best_H_loc = H_loc_candidate

    # Ora usiamo la H locale migliore trovata
    inliers = best_inliers
    H_loc = best_H_loc

    # ... (il resto della logica: calcolo shift, scelta H_use, calcolo ssim, etc. rimane identico) ...
    # ... (usa H_loc per calcolare lo shift, poi decidi se usare H_loc o H_glob) ...
    # Scegli la H migliore (locale o globale)
    use_local = (inliers >= MIN_INLIER_RATIO * K_NEIGHBORS) and (shift < 10) # Soglia fissa per lo shift
    H_use = H_loc if use_local else H_glob

    # Calcola SSIM per la qualità dell'allineamento
    rad_warped = cv2.warpPerspective(rad_img, H_use, (wO, hO))
    opt_patch_gray = cv2.cvtColor(crop_patch(opt_img, row.optical_x, row.optical_y, PATCH_SIZE), cv2.COLOR_BGR2GRAY)
    rad_patch = crop_patch(rad_warped, row.optical_x, row.optical_y, PATCH_SIZE)
    ssim_val = ssim(opt_patch_gray, rad_patch, data_range=rad_patch.max() - rad_patch.min())

    quality_metrics.append({
        'hole_id': row.hole_id, 'shift': shift, 'ssim': ssim_val,
        'inliers': inliers, 'chosen_H': "LOCAL" if use_local else "GLOBAL",
        **row.to_dict() # Aggiunge tutte le altre colonne
    })
    # Il resto del codice per calcolare shift, ssim e salvare le metriche rimane uguale
    # ...

metrics_df = pd.DataFrame(quality_metrics)
print("✅ Calcolo metriche con RANSAC adattivo completato.")

In [ ]:
# ===================================================================
# CELLA 4: CALCOLO OMOGRAFIA E METRICHE DI QUALITÀ(vecchia Soglia fissa)
# ===================================================================
print("--- [FASE 4] Calcolo Omografia e Metriche di Qualità ---")

# --- Carica dati ---
coord_df = pd.read_csv(MASTER_CSV_PATH)
opt_img = cv2.imread(str(OPTICAL_IMAGE_PATH))
rad_img = cv2.imread(str(NDT_IMAGE_PATH), 0)
hO, wO = opt_img.shape[:2]

src_pts = coord_df[['ndt_x', 'ndt_y']].to_numpy(np.float32)
dst_pts = coord_df[['optical_x', 'optical_y']].to_numpy(np.float32)

# --- Calcola H Globale una volta ---
H_glob, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
knn = NearestNeighbors(n_neighbors=K_NEIGHBORS).fit(src_pts)

quality_metrics = []
for i, row in tqdm(coord_df.iterrows(), total=len(coord_df), desc="Analizzando fori"):
    # Trova vicini e calcola H Locale
    src0 = src_pts[i].reshape(1,-1)
    _, indices = knn.kneighbors(src0)
    indices = indices.flatten()
    H_loc, mask = cv2.findHomography(src_pts[indices], dst_pts[indices], cv2.RANSAC, RANSAC_THRESHOLD)

    inliers = mask.ravel().sum() if mask is not None else 0

    # Calcola lo shift per H Locale
    proj = (H_loc @ np.append(src0,1).T).ravel(); proj /= proj[2]
    shift = np.hypot(proj[0] - row.optical_x, proj[1] - row.optical_y)

    # Scegli la H migliore (locale o globale)
    use_local = (inliers >= MIN_INLIER_RATIO * K_NEIGHBORS) and (shift < 10) # Soglia fissa per lo shift
    H_use = H_loc if use_local else H_glob

    # Calcola SSIM per la qualità dell'allineamento
    rad_warped = cv2.warpPerspective(rad_img, H_use, (wO, hO))
    opt_patch_gray = cv2.cvtColor(crop_patch(opt_img, row.optical_x, row.optical_y, PATCH_SIZE), cv2.COLOR_BGR2GRAY)
    rad_patch = crop_patch(rad_warped, row.optical_x, row.optical_y, PATCH_SIZE)
    ssim_val = ssim(opt_patch_gray, rad_patch, data_range=rad_patch.max() - rad_patch.min())

    quality_metrics.append({
        'hole_id': row.hole_id, 'shift': shift, 'ssim': ssim_val,
        'inliers': inliers, 'chosen_H': "LOCAL" if use_local else "GLOBAL",
        **row.to_dict() # Aggiunge tutte le altre colonne
    })

metrics_df = pd.DataFrame(quality_metrics)
print("✅ Calcolo metriche completato.")

In [ ]:
#### Cella 5.1 Filtraggio adattato dinamico con ransac sweep: Filtro di Qualità e Creazione `good_holes.csv`

'''Questa cella prende le metriche, applica un filtro intelligente e salva la lista dei fori che verranno usati per il dataset finale.
'''
# ===================================================================
# CELLA 5: FILTRO DI QUALITÀ
# ===================================================================
print("--- [FASE 5] Filtraggio dei fori di alta qualità ---")

## --- Calcola soglie dinamiche basate sui quantili ---
# Vogliamo tenere circa l'80% dei fori migliori
#**Perché funziona:** Invece di usare soglie fisse che potrebbero essere troppo severe, usiamo i quantili. Questo approccio ci garantisce di tenere una **percentuale fissa** dei migliori allineamenti. Se impostiamo il quantile a 0.80, stiamo dicendo: "Prendi l'80% dei fori con lo shift più basso e l'80% dei fori con l'SSIM più alto". Questo ci dà un controllo diretto sulla dimensione del dataset finale.

SHIFT_MAX = metrics_df['shift'].quantile(0.80) # Scarta il 20% peggiore per shift
SSIM_MIN = metrics_df['ssim'].quantile(0.20)  # Scarta il 20% peggiore per SSIM
#dinamico fine- continua normale adesso
print(f"Soglie dinamiche calcolate: Shift <= {SHIFT_MAX:.2f}, SSIM >= {SSIM_MIN:.2f}")

# --- Applica filtro ---
good_mask = (
    (metrics_df['shift'] <= SHIFT_MAX) &
    (metrics_df['ssim'] >= SSIM_MIN) &
    (metrics_df['inliers'] >= MIN_INLIER_RATIO * K_NEIGHBORS)
)
good_holes_df = metrics_df[good_mask]

GOOD_HOLES_CSV_PATH = OUTPUT_BASE_DIR / f"quality_{SCAN_ID}.csv"
good_holes_df.to_csv(GOOD_HOLES_CSV_PATH, index=False)

print(f"✅ Fori di alta qualità filtrati: {len(good_holes_df)} / {len(metrics_df)}. Salvati in {GOOD_HOLES_CSV_PATH}")

In [ ]:
#### Cella 6: Generazione Finale delle Patch e ZIP

'''L'ultima cella. Legge `good_holes.csv` e crea le immagini, le maschere, gli overlay e lo zip finale.
'''
# ===================================================================
# CELLA 6: GENERAZIONE PATCH FINALI E ZIP
# ===================================================================
print("--- [FASE 6] Generazione Patch e Packaging ---")

good_holes_df = pd.read_csv(GOOD_HOLES_CSV_PATH)

for i, row in tqdm(good_holes_df.iterrows(), total=len(good_holes_df), desc="Creando patch finali"):
    H_use = H_glob # Per semplicità usiamo sempre la H globale per il warp finale, ma potremmo ricalcolare H_loc

    rad_warped = cv2.warpPerspective(rad_img, H_use, (wO, hO))

    opt_patch = crop_patch(opt_img, row.optical_x, row.optical_y, PATCH_SIZE)
    rad_patch = crop_patch(rad_warped, row.optical_x, row.optical_y, PATCH_SIZE)

    # Crea la maschera a 3 classi (0, 1, 2)
    final_mask = create_ground_truth_mask(rad_patch)

    # Salva immagine e maschera
    filename_base = f"hole_{int(row.hole_id):04d}"
    cv2.imwrite(str(OUTPUT_BASE_DIR / "final_patches/images" / f"{filename_base}.png"), opt_patch)
    cv2.imwrite(str(OUTPUT_BASE_DIR / "final_patches/masks" / f"{filename_base}.png"), final_mask)

    # Crea e salva overlay di debug
    rad_bgr = cv2.cvtColor(rad_patch, cv2.COLOR_GRAY2BGR)
    overlay = cv2.addWeighted(opt_patch, 0.6, rad_bgr, 0.4, 0)
    cv2.imwrite(str(OUTPUT_BASE_DIR / "overlays" / f"{filename_base}_overlay.png"), overlay)

# --- Packaging ---
ZIP_PATH = OUTPUT_BASE_DIR / f"unet_dataset_{SCAN_ID}.zip"
with zipfile.ZipFile(ZIP_PATH, 'w', zipfile.ZIP_DEFLATED) as zf:
    for folder in ["images", "masks"]:
        for file_path in (OUTPUT_BASE_DIR / "final_patches" / folder).glob("*.png"):
            zf.write(file_path, f"{folder}/{file_path.name}")

print(f"\n✅ Processo completato! Dataset pronto per il training in '{ZIP_PATH}'")

In [ ]:
##codice vecchio##

Calcolo fori e fattore di scala

In [ ]:
#@title Per statistiche/Histogramma Scale ## %% 7 – Scale, overlay e statistiche preliminari controlla coerenza scans
# %% 7 – Scale, overlay e statistiche preliminari
# • Definisce percorsi CSV master, immagini full scan e directory overlay
# • Crea directory per overlay per foro
# • Parametri di rough‑crop e fallback diametri
# • Carica DataFrame di coordinate (index=hole_id), ottica e radio full
# • Loop su ogni foro con tqdm:
#     – rough‑crop ottico e radio per calcolare diametri
#     – funzione diam(): binarizza, chiude, trova contorno, fitEllipse → diametro medio
#     – usa fallback se nessun contorno trovato
#     – calcola scale S = diam_opt/diam_rad
#     – aggiunge record a lista rows
#     – crea overlay per debug: ridimensiona radio con S, applica colormap jet, alpha‑blend con ottica
#     – salva overlay PNG in overlay_per_hole
# • Alla fine scrive CSV scale_per_hole.csv con hole_id, diam_opt, diam_rad, scale_S
# • Plotta istogramma di scale_S con linea media

# Config
CSV_C    = "/content/Homography_Output_1A/1A_master.csv"
OPT_SCAN = OPTICAL_IMAGE_PATH
RAD_SCAN = NDT_IMAGE_PATH
OUT_S    = "scale_per_hole.csv"
OVL_DIR  = Path("overlay_per_hole"); OVL_DIR.mkdir(exist_ok=True)
CROP_O   = 400; CROP_R = 200
FALL_O   = 283; FALL_R = 142
ALPHA=0.4
df = pd.read_csv(CSV_C).set_index('hole_id')
opt_full = cv2.imread(OPT_SCAN)
rad_full = cv2.imread(RAD_SCAN,0)

rows = []
for hid, r in tqdm(df.iterrows(), total=len(df)):
    ox, oy, rx, ry = r.optical_x, r.optical_y, r.ndt_x, r.ndt_y
    def diam(patch, fallback):
        cnts, _ = cv2.findContours(
            cv2.threshold(
                cv2.cvtColor(patch,cv2.COLOR_BGR2GRAY)
                if patch.ndim==3 else patch,
                0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU
            )[1],
            cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        if not cnts: return fallback
        (_, _),(MA,ma),_ = cv2.fitEllipse(max(cnts, key=cv2.contourArea))
        return (MA+ma)/2

    d_o = diam(crop_patch(opt_full,ox,oy,CROP_O), FALL_O)
    d_r = diam(crop_patch(rad_full,rx,ry,CROP_R), FALL_R)
    S   = round(d_o/d_r,4)
    rows.append([hid, round(d_o,2), round(d_r,2), S])

    # overlay debug
    opt_p = crop_patch(opt_full,ox,oy,CROP_O)
    rad_p = cv2.resize(crop_patch(rad_full,rx,ry,CROP_R),
                       None, fx=S, fy=S, interpolation=cv2.INTER_LINEAR)
    rad_p = cv2.resize(rad_p,(CROP_O,CROP_O))
    cm = cv2.applyColorMap(
        cv2.normalize(rad_p,None,0,255,cv2.NORM_MINMAX).astype('uint8'),
        cv2.COLORMAP_JET
    )
    ov = cv2.addWeighted(opt_p,1-ALPHA,cm,ALPHA,0)
    cv2.imwrite(str(OVL_DIR/f"{hid:03d}_ovl.png"), ov)

with open(OUT_S,'w',newline='') as f:
    csv.writer(f).writerows([['hole_id','diam_opt_px','diam_rad_px','scale_S']]+rows)

print(f"✓ Celle 7 eseguite: CSV in {OUT_S}, overlay in {OVL_DIR}")
plt.hist([r[3] for r in rows], bins=15)
plt.title("Distribuzione scale_S")
plt.show()
'''
Cosa fa
1. Ritaglia un rough crop_patch di ciascun foro su ottica (CROP_O) e radio (CROP_R).
2. Binarizza, chiude, fitEllipse → diametro medio.
3. scale_S = diam_opt / diam_rad → fattore di scala geometrico.
4. Genera overlay JET (debug visivo) senza omografia, solo riscalando radialmente.
5. Scrive scale_per_hole.csv; fa istogramma e z‑score.

Quando ti serve
* Per capire se la radiografia intera è già grossomodo scalata come l’ottica.
* Per individuare outlier geometrici (fori deformi o segmentati male) prima di investire tempo in omografie locali.

    Non è collegata ad H locali/globali; funziona “a monte”.
'''

In [ ]:
#@title %% 8 – Analisi z‑score su scale_S
# %% 8 – Analisi z‑score su scale_S
# • Carica CSV scale_per_hole.csv
# • Estrae vettore S delle scale per foro
# • Calcola media e deviazione standard
# • Aggiunge colonna z_score = (S−mean)/std
# • Stampa media, σ e eventuali outlier con |z|>2
# • Plotta istogramma di S con linea punteggiata alla media

dfz = pd.read_csv("scale_per_hole.csv")
S = dfz['scale_S']
mean, std = S.mean(), S.std()
dfz['z_score'] = (S-mean)/std

print(f"Media S̄ = {mean:.4f}, σ = {std:.4f}")
out = dfz[dfz.z_score.abs()>2]
if not out.empty:
    print("Outlier (|z|>2):")
    print(out[['hole_id','scale_S','z_score']])

plt.hist(S, bins=15)
plt.axvline(mean, linestyle='--')
plt.title("z‑score delle scale")
plt.show()


#Che cosa fa la cella 12 riga‑per‑riga
<details>
    Crea/assicia cartella OUT = Path("/content/out_overlay").

    Per ogni foro

        calcola H_loc (K = 9 + RANSAC 3 px);

        stima lo shift di quel foro ⇒ decide LOCAL vs GLOBAL;

        warpa la radiografia intera con H_use;

        ritaglia patch ottico & radio col tuo crop() (garantito 704×704);

        colormap + alpha‑blend → overlay_${hole}.png;

        calcola SSIM e accumula in rows.

    Alla fine (cella 13) salva:

df = pd.DataFrame(rows)                    # DataFrame vero
df.to_csv(OUT / "quality.csv", index=False)

e stampa l’elenco “⚠️ Fori da controllare” (shift > 5 px o SSIM < 0.30)

In [ ]:
#@title (CORE) %% 12  – Loop principale per omografia locale e overlay qualità. Loop locale vs globale + overlay & SSIM	USA (core)	• Usa H_glob, nn, QUALITY_THR, MIN_INLIERS, CROP.
#• Salva overlay e popola rows
rows = []
from pathlib import Path
OUT = Path("/content/out_overlay")    # <‑‑ adesso è Path
OUT.mkdir(exist_ok=True)              # cartella se non esiste
for i,row in coord.iterrows():
    src0 = src[i].reshape(1,-1)
    _,idx = nn.kneighbors(src0); idx = idx.flatten()

    H_loc, msk = cv2.findHomography(src[idx], dst[idx], cv2.RANSAC, 3)
    inl = int(msk.sum()) if msk is not None else 0

    # stima shift del centro
    proj  = (H_loc @ np.append(src0,1).T).ravel(); proj /= proj[2]
    shift = np.hypot(proj[0]-row.optical_x, proj[1]-row.optical_y)

    use_local = inl>=MIN_INLIERS and shift<=QUALITY_THR
    H_use = H_loc if use_local else H_glob
    tag   = "LOCAL" if use_local else "GLOBAL"

    # ---- ritaglia ROI ottica ----------------------------
    cx,cy = int(row.optical_x), int(row.optical_y)
    x1,y1 = cx-CROP//2, cy-CROP//2 ; x2,y2 = x1+CROP, y1+CROP
    opt_patch = crop(opt_img, cx, cy, CROP)

    # ---- warp radiografia SOLO per quest’area -----------
    #   trucco: applichiamo H_use all’intera immagine, è <0.1 s con 8 MP
    rad_warp = cv2.warpPerspective(rad_raw, H_use, (wO,hO))
    rad_patch = crop(rad_warp, cx, cy, CROP)     # garantisce dimensione CROP×CROP


    # ---- overlay + colormap -----------------------------
    rad_col = cv2.applyColorMap(cv2.normalize(rad_patch,None,0,255,cv2.NORM_MINMAX)
                                .astype('uint8'), cv2.COLORMAP_JET) # docs :contentReference[oaicite:3]{index=3}
    over = cv2.addWeighted(opt_patch, 1-ALPHA, rad_col, ALPHA, 0)

    # ---- SSIM (qualità texture) -------------------------
    #ssim_val = ssim(cv2.cvtColor(opt_patch,cv2.COLOR_BGR2GRAY), rad_patch,
    #                data_range=rad_patch.ptp())            # docs :contentReference[oaicite:4]{index=4}
    # choice A – use the functional call (clean & portable)
    ssim_val = ssim(cv2.cvtColor(opt_patch, cv2.COLOR_BGR2GRAY),
                    rad_patch,
                    data_range=np.ptp(rad_patch))          # <- np.ptp !
    # choice B – spell it out, no ptp() at all
    """ssim_val = ssim(cv2.cvtColor(opt_patch, cv2.COLOR_BGR2GRAY),
                rad_patch,
                data_range=rad_patch.max() - rad_patch.min())"""
    cv2.putText(over, f"id:{row.hole_id}  shift:{shift:.2f}px  SSIM:{ssim_val:.3f}  {tag}",
                (10,30), cv2.FONT_HERSHEY_SIMPLEX, 0.7,(255,255,255),2,cv2.LINE_AA)
    cv2.imwrite(str(OUT/f"overlay_{row.hole_id:04d}.png"), over)

    rows.append(dict(hole=row.hole_id, shift=shift, ssim=ssim_val,
                     inliers=inl, chosen=tag,optical_x=row.optical_x, optical_y=row.optical_y,
                     ndt_x=row.ndt_x, ndt_y=row.ndt_y))

df_metrics = pd.DataFrame(rows)

In [ ]:
# %% 12‑bis – calcolo soglie auto POI SUBITO DOPO 13 PER IL FILTRAGGIO

df = pd.DataFrame(rows)
df_metrics = pd.DataFrame(rows)

SHIFT_MAX   = 2 * df_metrics['shift'].median()                     #  ≈ 2·mediana
SSIM_MIN    = max(0.15, df_metrics['ssim'].mean()
                         - 2*df_metrics['ssim'].std())             #  mai sotto 0.15
MIN_INLIERS = int(round(0.7 * 9))                                  #  70 % di 9
#Nota: non ridefinire più QUALITY_THR, SSIM_THR ecc.; usa queste tre nuove costanti (SHIFT_MAX, SSIM_MIN, MIN_INLIERS) in tutte le celle successive.
print(f"Soglie → shift ≤ {SHIFT_MAX:.1f} px   SSIM ≥ {SSIM_MIN:.3f}   inliers ≥ {MIN_INLIERS}")
df.to_csv(OUT / "quality.csv", index=False)

print("written:", (OUT / "quality.csv").exists())
bad = [r['hole'] for r in rows if r['shift']>QUALITY_THR or r['ssim']<.3]
print("⚠️  Fori da controllare:", bad)

In [ ]:
import matplotlib.pyplot as plt
dfm = pd.DataFrame(rows)

fig, axs = plt.subplots(1,3, figsize=(12,3))
axs[0].hist(dfm['shift'], bins=20); axs[0].set_title("shift [px]")
axs[1].hist(dfm['ssim'],  bins=20); axs[1].set_title("SSIM")
axs[2].hist(dfm['inliers'], bins=10); axs[2].set_title("inlier K={VAR}")
plt.show()


In [ ]:
# %% 13 – quality.csv + good_holes.csv
df_metrics['good'] = (df_metrics['shift']  <= SHIFT_MAX) & \
                     (df_metrics['ssim']   >= SSIM_MIN)  & \
                     (df_metrics['inliers']>= MIN_INLIERS)

df_metrics.to_csv(OUT / "quality.csv", index=False)

good = df_metrics[df_metrics['good']]
print(f"Fori OK: {len(good)}/{len(df_metrics)}")

good_holes = (good[['hole','chosen']]          # hole_id + tag LOCAL/GLOBAL
              .rename(columns={'hole':'hole_id'})
              .merge(coord_df, on='hole_id'))  # aggiunge tutte le coordinate

good_holes.to_csv("good_holes.csv", index=False)


In [ ]:
# df_metrics = DataFrame già calcolato (hole, shift, ssim, inliers, chosen…)

MIN_OK      = 0.9          # voglio almeno il 70 % di fori buoni
MAX_ITER    = 100           # non iterare all’infinito
K           = 6            # vicini usati per H_loc

for it in range(MAX_ITER):
    # 1) calcola soglie a partire dai quantili attuali
    SHIFT_MAX  = df_metrics['shift'].quantile(0.90)      # scarta 10 % peggiore
    SSIM_MIN   = df_metrics['ssim' ].quantile(0.1)      # scarta 10 % peggiore
    INL_MIN    = int(round(0.7 * K))                     # 70 % di inlier

    # 2) etichetta i “good”
    good_mask = (df_metrics['shift']  <= SHIFT_MAX) & \
                (df_metrics['ssim']   >= SSIM_MIN)  & \
                (df_metrics['inliers']>= INL_MIN)
    good_pct  = good_mask.mean()           # frazione [0–1]

    print(f"[iter {it}] ok {good_pct:.2%}  "
          f"shift≤{SHIFT_MAX:.1f}  SSIM≥{SSIM_MIN:.3f}")

    # 3) se basta, esci
    if good_pct >= MIN_OK:
        break

    # 4) altrimenti allenta progressivamente (p. es. alza quantili)
    #    qui un esempio molto semplice:
    df_metrics.loc[~good_mask, 'shift']  *= 0.9   # riduco un po' gli shift outlier
    df_metrics.loc[~good_mask, 'ssim']   *= 1.05  # aumento leggermente SSIM outlier
    # (puoi elaborare logiche più sofisticate)


In [ ]:
# good_mask è già definito nel punto precedente
df_metrics['good'] = good_mask
df_metrics.to_csv(OUT / "quality.csv", index=False)

# ── crea il DataFrame BUONO ────────────────────────────────
good_holes = (df_metrics[df_metrics['good']]           # tieni solo i buoni
              [['hole', 'chosen']]                     # id + LOCAL/GLOBAL
              .rename(columns={'hole': 'hole_id'})     # uniforma nome
              .merge(coord_df, on='hole_id'))          # aggiungi coordinate

good_holes.to_csv("good_holes.csv", index=False)
print("good_holes.csv salvato, fori OK:", len(good_holes))
print(good_holes.head())
print(good_holes.columns)


In [ ]:
#@title Avanzato; alla fine; fine tune %% Ransac sweep overlay - overlay critici - report # modifica src
#Scopo : trovare a posteriori il threshold RANSAC che massimizza l’inlier‑ratio su tutti i punti — e produrre overlay dei “fori critici” con ogni soglia.
#* Imposta ransac_thresholds = np.arange(1.0, 10.1, 0.5) prima del loop.
#* Puoi riutilizzarla dopo aver visto che la soglia 5 px della cella 5a non è ottimale.
#* Una volta trovato best_H, sostituisci H_glob con best_H e rigenera overlay/patch senza rifare tutto il notebook
# --- file già prodotti nel tuo notebook -----------------------------
CSV_COORD   = "/content/T0_90_1A_master.csv"     # ottico_x/y e ndt_x/y
OPT_SCAN    = "/content/T0_90_1A_ingresso.jpg"
RAD_SCAN    = "/content/Carbon Textile 1A.jpg"

# Carica le immagini full e le coordinate una volta
opt_full = cv2.imread(OPT_SCAN)                     # BGR
rad_full = cv2.imread(RAD_SCAN, 0)                  # grey
coord_df = pd.read_csv(CSV_COORD)

# Estrai i punti sorgente (radio/NDT) e destinazione (ottico)
src_points = np.float64(coord_df[['ndt_x', 'ndt_y']].values)
dst_points = np.float64(coord_df[['optical_x', 'optical_y']].values)

# Define critical hole IDs and overlay parameters
critical_hole_ids = [1, 2, 61, 62, 63, 64]
ALPHA = 0.40
CROP_SIZE = 300

# Create output directory for overlays
OVERLAY_DIR_BASE = "/content/overlays_per_threshold"
os.makedirs(OVERLAY_DIR_BASE, exist_ok=True)

# Initialize tracking variables
best_inlier_ratio = 0
best_threshold = None
best_H = None

# Prepare report data
report_data = []

# Start the loop through RANSAC thresholds
for threshold in tqdm(ransac_thresholds, desc="Processing thresholds"):
    # Calculate Homography for the current threshold
    H, mask = cv2.findHomography(src_points, dst_points, cv2.RANSAC, threshold, maxIters=10000)

    if mask is not None:
        inlier_ratio = mask.mean()
    else:
        inlier_ratio = 0
        H = None # Ensure H is None if mask is None

    # Update best H and threshold if current inlier ratio is higher
    if inlier_ratio > best_inlier_ratio:
        best_inlier_ratio = inlier_ratio
        best_threshold = threshold
        best_H = H.copy() if H is not None else None # Store a copy of H

    # Calculate metrics for the current threshold
    detH = np.linalg.det(H) if H is not None else np.nan
    scale_x = np.linalg.norm(H[0,:2]) if H is not None else np.nan
    scale_y = np.linalg.norm(H[1,:2]) if H is not None else np.nan

    # Store report data
    report_data.append({
        "threshold": threshold,
        "inlier_ratio": inlier_ratio,
        "detH": detH,
        "scale_x": scale_x,
        "scale_y": scale_y
    })

    # Generate overlays for critical holes
    if H is not None:
        h_opt, w_opt = opt_full.shape[:2]
        rad_warped = cv2.warpPerspective(rad_full, H, (w_opt, h_opt))

        overlay_subdir = os.path.join(OVERLAY_DIR_BASE, f"threshold_{threshold:.1f}")
        os.makedirs(overlay_subdir, exist_ok=True)

        for hid in critical_hole_ids:
            if hid not in coord_df['hole_id'].values:
                print(f"Warning: Hole ID {hid} not found in coordinates CSV. Skipping overlay.")
                continue

            # Find the coordinates for the hole
            hole_coords = coord_df[coord_df['hole_id'] == hid].iloc[0]
            ox, oy = int(hole_coords.optical_x), int(hole_coords.optical_y)

            # Crop patches from the full images
            opt_patch = crop(opt_full, ox, oy, CROP_SIZE)
            rad_patch_aligned = crop(rad_warped, ox, oy, CROP_SIZE)

            # Create and save overlay
            rad_patch_color = cv2.cvtColor(rad_patch_aligned, cv2.COLOR_GRAY2BGR)
            if opt_patch.shape != rad_patch_color.shape:
                 print(f"Warning: Patch shapes mismatch for hole {hid}: {opt_patch.shape} vs {rad_patch_color.shape}. Skipping overlay.")
                 continue
            overlay = cv2.addWeighted(opt_patch, 1 - ALPHA, rad_patch_color, ALPHA, 0)
            cv2.imwrite(os.path.join(overlay_subdir, f"overlay_{hid}.png"), overlay)

# After the loop, print the best threshold and generate the full report
print(f"\nBest threshold found: {best_threshold:.1f} with inlier ratio: {best_inlier_ratio:.2%}")

report_df = pd.DataFrame(report_data)
print("\nDetailed Report:")
display(report_df)

# Optional: Save the report to a CSV file
report_csv_path = "/content/ransac_threshold_report.csv"
report_df.to_csv(report_csv_path, index=False)
print(f"\nDetailed report saved to: {report_csv_path}")

# Note: The best_H matrix is stored in the variable best_H

In [ ]:
#@title esegui su fori SSIM <0.3 ## %% Check angolo su quality – Verifica angoli (fallback B)
#• Usa CSV_PATH e NON hard‑code altri CSV.
# %%  – Verifica angoli (fallback B)
# • Definisce parametri di crop per ottica e radio, intervallo angoli e passo
# • Inizializza CSV ang_check.csv con header hole_id, best_angle, xor_loss
# • Carica DataFrame dei fori, immagine ottica full e radiografia full
# • Definisce mask_img() per binarizzare e smoothare con Otsu+medianBlur
# • Definisce xor_loss() per misurare differenza bit‑wise tra maschere
# • Loop su ogni foro:
#     – estrae patch grezze ottica e radio
#     – calcola maschera ottica
#     – per ogni angolo in [-Δ,Δ] con passo STEP:
#         » ruota patch radio
#         » calcola xor_loss con maschera ottica
#         » tiene il migliore (min loss)
#     – scrive sul CSV hole_id, best_angle e loss minimo

# Parametri per angolo
PATCH_O = 704; PATCH_R = 256
DELTA   = 12.5; STEP   = 0.1
CSV_IN  = "/content/T0_90_1A_master.csv"
CSV_OUT = "ang_check.csv"

# Init CSV
with open(CSV_OUT, 'w', newline='') as f:

  csv.writer(f).writerow(["hole_id","best_angle_deg","xor_loss"])

df_in = pd.read_csv(CSV_IN)
opt = cv2.imread(OPTICAL_FULL_PATH)
rad = cv2.imread(NDT_FULL_PATH, 0)

def mask_img(im):
    g = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY) if im.ndim==3 else im
    _,th = cv2.threshold(g,0,255,cv2.THRESH_OTSU)
    return cv2.medianBlur(th,5)

def xor_loss(a,b):
    h,w = min(a.shape[0],b.shape[0]), min(a.shape[1],b.shape[1])
    return cv2.countNonZero(cv2.bitwise_xor(a[:h,:w], b[:h,:w]))

for _,r in df_in.iterrows():
    hid, ox, oy, rx, ry = r.hole_id, r.optical_x, r.optical_y, r.ndt_x, r.ndt_y
    o_p = crop(opt, ox, oy, PATCH_O)
    r_p = crop(rad, rx, ry, PATCH_R)
    m_o = mask_img(o_p)

    best_l, best_a = 1e9, 0
    for ang in np.arange(-DELTA, DELTA+STEP, STEP):
        M = cv2.getRotationMatrix2D((PATCH_R/2,PATCH_R/2), ang, 1)
        rot = cv2.warpAffine(r_p, M, (PATCH_R,PATCH_R),
                             flags=cv2.INTER_LINEAR,
                             borderMode=cv2.BORDER_CONSTANT)
        loss = xor_loss(m_o, mask_img(rot))
        if loss < best_l:
            best_l, best_a = loss, ang

    with open(CSV_OUT,'a',newline='') as f:
        csv.writer(f).writerow([hid, round(best_a,2), int(best_l)])

print(f"✓ Celle 6 eseguite: angoli in {CSV_OUT}")


In [ ]:
# %% 25a – fori sospetti da quality.csv
qual = pd.read_csv(OUT / "quality.csv")
suspect = qual.loc[
            (qual['shift'] > QUALITY_THR) |
            (qual['ssim']  < SSIM_THR), 'hole']
suspect.to_csv("suspect.csv", index=False, header=['hole_id'])

# %% 6 – angle‑check sui soli fori sospetti
CSV_IN  = "suspect.csv"     # <‑‑ adesso NON hard‑code
CSV_OUT = "ang_check.csv"


DEBUG creazioe zip overlays

In [ ]:
# %% 26 – Patch finali + ZIP (allineate)
from pathlib import Path
import cv2, zipfile, numpy as np, pandas as pd
from tqdm.auto import tqdm

PATCH_DIR   = Path("/content/final_patches"); PATCH_DIR.mkdir(exist_ok=True)
PATCH_SIZE  = 704
ALPHA       = 0.4

# 1) carica dati
full      = pd.read_csv("good_holes.csv")           # ⬅️ creato dopo quality.csv
angles    = pd.read_csv("ang_check.csv").set_index('hole_id')['best_angle_deg'] if Path("ang_check.csv").exists() else {}
opt_full  = cv2.imread(OPTICAL_FULL_PATH)
rad_full  = cv2.imread(NDT_FULL_PATH, 0)
hO, wO    = opt_full.shape[:2]

# 2) opzionale: dizionario H_loc salvati durante la cella 12
#    se non l'hai salvato, ricomputalo al volo con la stessa logica
H_loc_dict = {}   # {hole_id: H_loc}

for _, r in tqdm(full.iterrows(), total=len(full), desc="patch"):
    hid = int(r.hole_id)
    # --- H da usare --------------------
    if r.chosen == "LOCAL":
        H_use = H_loc_dict.get(hid, H_glob)   # fallback safetynet
    else:
        H_use = H_glob

    # --- warp e crop -------------------
    rad_warp = cv2.warpPerspective(rad_full, H_use, (wO, hO))
    cx, cy   = int(r.optical_x), int(r.optical_y)
    opt_p    = crop(opt_full, cx, cy, PATCH_SIZE)
    rad_p    = crop(rad_warp, cx, cy, PATCH_SIZE)

    # --- rotazione fine ----------------
    ang = angles.get(hid, 0.0)
    if abs(ang) > 0.1:
        M = cv2.getRotationMatrix2D((PATCH_SIZE/2, PATCH_SIZE/2), ang, 1.0)
        rad_p = cv2.warpAffine(rad_p, M, (PATCH_SIZE, PATCH_SIZE),
                               flags=cv2.INTER_LINEAR,
                               borderMode=cv2.BORDER_CONSTANT, borderValue=0)

    # --- salva -------------------------
    cv2.imwrite(str(PATCH_DIR/f"{hid:03d}_opt.png"), opt_p)
    cv2.imwrite(str(PATCH_DIR/f"{hid:03d}_msk.png"), rad_p)
    # >>> GARANZIA SHAPE <<< ---------------------------------
    if rad_p.shape[:2] != opt_p.shape[:2]:
      print(f"⚠️  {hid:03d}   mismatch {rad_p.shape} vs {opt_p.shape}")
      h,w   = opt_p.shape[:2]
      rad_p = cv2.resize(rad_p, (w, h), interpolation=cv2.INTER_NEAREST)
    rad_bgr = cv2.cvtColor(rad_p, cv2.COLOR_GRAY2BGR)
    ovl = cv2.addWeighted(opt_p, 1-ALPHA,
                          cv2.cvtColor(rad_p, cv2.COLOR_GRAY2BGR),
                          ALPHA, 0)
    cv2.imwrite(str(PATCH_DIR/f"{hid:03d}_ovl.png"), ovl)

# 3) zip
zip_path = "/content/patches_unet_ready.zip"
with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zf:
    for p in PATCH_DIR.glob("*.png"):
        zf.write(p, p.name)

print("✔ Patch pronte e zip create:", zip_path)


In [ ]:
# %% FULL‑SCAN OVERLAY – una sola riga di output
# ①  warp dell’intera radiografia con l’omografia scelta
rad_full_warp = cv2.warpPerspective(rad_full, H_glob, (wO, hO))

# ②  alpha‑blend: 60 % ottica + 40 % radio colore (colormap JET o bordi Canny)
rad_color = cv2.applyColorMap(
               cv2.normalize(rad_full_warp, None, 0, 255, cv2.NORM_MINMAX)
               .astype("uint8"),
               cv2.COLORMAP_JET)
full_overlay = cv2.addWeighted(opt_full, 0.6, rad_color, 0.4, 0)

# ③  salva il PNG
cv2.imwrite("/content/full_scan_overlay.png", full_overlay)
print("✅ full_scan_overlay.png creato")


aggiustamento visivo omografia

In [ ]:
# calcola shift locale usando H_glob già stimata
#Se vedi due blocchi con errori diversi (sinistra bassa, destra alta) ⇒ prova una omografia per metà pannello (vedi punto 3).
#
#Se l’errore cresce gradualmente lungo X o Y ⇒ hai una componente affine non modellata (scala non uniforme).
# calcola shift_glob UNA volta
err=[]
for _,r in coord_df.iterrows():
    est = (H_glob @ [r.ndt_x, r.ndt_y, 1]).ravel(); est/=est[2]
    err.append(np.hypot(est[0]-r.optical_x, est[1]-r.optical_y))
coord_df['shift_glob']=err

pivot = coord_df.pivot(index='optical_y', columns='optical_x',
                       values='shift_glob')
mat = pivot.sort_index(ascending=False).values  # capovolgi y

plt.imshow(mat, cmap='magma_r', vmax=10, aspect='auto')
plt.colorbar(label='shift [px]')
plt.title("Heat‑map errore H_glob"); plt.show()


| colore (con `magma_r`) | shift (px) | lettura pratica                                                 |
| ---------------------- | ---------- | --------------------------------------------------------------- |
| chiaro / giallo        | ≈ 0 – 2    | allineamento ottimo                                             |
| arancio                | 3 – 7      | piccoli errori, micro‑refine o H\_loc li risolve                |
| rosso‑scuro            | 8 – 15     | H\_glob non basta, serve H\_loc (o sotto‑pannello)              |
| quasi nero (cappuccio) | > 15       | punti chiaramente fuori (scansione doppia, csv invertito, ecc.) |

Pattern a blocchi (sinistra buona, destra cattiva) → calcola
due H_glob (pannelli).

Gradiente regolare (da sinistra a destra) → scala non uniforme: prova cv2.getAffineTransform locale o aumenta n_neighbors.

Sporadici punti rossi → tienili in fallback GLOBAL e usa il micro‑refine.

In [ ]:
plt.scatter(coord_df.ndt_x, coord_df.ndt_y, s=5, label='NDT')
plt.scatter(coord_df.optical_x, coord_df.optical_y, s=5, label='Optical')
plt.gca().invert_yaxis(); plt.legend(); plt.show()


Check allineamento per via di omografia problematica

In [ ]:
print("NDT rows:", len(ndt_df), "Optical rows:", len(opt_df))
#Se i numeri differiscono, non stai comparando le stesse entità.
merged = opt_df.merge(ndt_df, on='hole_idx', how='inner')
print("buchi in comune:", len(merged))
#Se “buchi in comune” è ≪ 64 vuol dire che alcuni ID mancano da una parte.